# ETL Pipeline

The Goal of this script is to create a non-relational database and ETL pipeline for a music streaming app. 
Its tables should be able to answer queries with in the following form:   
* Give me the artist, song title and song's length in the music app history that was heard   
during  sessionId = 338, and itemInSession  = 4
* Give me only the following: name of artist, song (sorted by itemInSession) and user (first   
and last name) for userid = 10, sessionid = 182
* Give me every user name (first and last) in my music app history who listened to the song    
'All Hands Against His Own'   


## #1 Preparing Files

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from typing import Iterator, Dict, Any, Optional

In [3]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [4]:
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
             
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

#print(len(full_data_rows_list)) # total number of rows 
#print(full_data_rows_list) # preview list of data rows
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## #2 Preparing Keyspace

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)
    
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Handling queries efficiently

In [7]:
def cql(query: str) -> None:
    """Query handling"""
    try:
        session.execute(query)
    except Exception as e:
        print(e)

## #3 Creating Tables and Processing Data

In the following sections I will design a table to answer one of the queries mentioned at the beginning.
Therefore the following steps apply:
* translating the questions into cql
* designing tables on the basis of those queries
* inserting the data from csv into those tables
* validating the data by answering the example tasks mentioned above.


### Query1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Translation in to CQL:
* `SELECT sessionId, itemInSession, artist, song, length FROM session_songs   
  WHERE sessionId = 338 AND itemInSession = 4 LIMIT 10`


Description of the query
* `sessionId` and `itemInSession` are included in the WHERE statement therefore they must be part of the PRIMARY KEY.   
* together they make a row unique --> no additional columns needed inside the PRIMARY KEY
* if only `sessionId` would be partition key that could lead to an uneven distribution as some sessions may be longer than others
* therefore both of them should serve as partition key
* `sessionId` and `itemInSession` must come first in the CREATE, INSERT and SELECT statements,   
  otherwise it will have consequences for the READ performance.

In [15]:
# create table
cql("""DROP TABLE IF EXISTS session_songs; """)
cql("""CREATE TABLE session_songs (
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    length DOUBLE,
    PRIMARY KEY ((sessionId, itemInSession))
    )""")



# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO session_songs (sessionId, itemInSession, artist, song, length)
                VALUES (%s, %s, %s, %s, %s)"""
        #Each row read from the csv file is returned as a list of strings No automatic data type conversion is performed
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

        
    
# validate data
#code integrated from here: https://stackoverflow.com/a/41484806/13560354
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

## The format to return row results. By default, each returned row will be a named tuple.
session.row_factory = pandas_factory

##By default, this many rows will be fetched at a time. Setting this to None will disable automatic paging for large query results
session.default_fetch_size = None

query = """SELECT sessionId, itemInSession, artist, song, length 
        FROM session_songs WHERE sessionId = 338 AND itemInSession = 4 LIMIT 10"""
results = session.execute(query, timeout=None)

##The list of current page rows. May be empty if the result was empty, or this is the last page.
df1 = results._current_rows
df1[['artist', 'song', 'length']].head()


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Translation in to CQL:
* `SELECT sessionId, userId, itemInSession, artist, song,   
  firstName, lastName FROM user_session_songs   
  WHERE sessionId = 182 AND userId = 10`

* with `sessionId` and `userId` as partition key   
    and `itemInSession` as clustering column


Description of the query
* `sessionId` and `userId` are included in the WHERE statement therefore they must be part of the PRIMARY KEY   
* The result has to be ordered by `itemInSession`, therefore this will become the clustering column
* `sessionId, userId, itemInSession` make a row unique --> no additional columns needed for the PRIMARY KEY
* Again, if only `sessionId` would be partition key that could lead to an uneven distribution as some sessions may be longer than others
* Therefore `sessionId` and `userId` together serve as partition key for a more even distribution

* `sessionId, userId, itemInSession` must come first in the CREATE, INSERT and SELECT statements,   
  as otherwise it will have consequences for the READ performance

In [31]:
# create table
cql("""DROP TABLE IF EXISTS user_session_songs; """)
cql("""CREATE TABLE user_session_songs (
    sessionId INT,
    userId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY ((sessionId, userId), itemInSession))
    """)


# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_session_songs (sessionId, userId, itemInSession, artist, song, firstName, lastName)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

        
# validate data
session.row_factory = pandas_factory
session.default_fetch_size = None
query = """SELECT sessionId, userId, itemInSession, artist, song, firstName, lastName
        FROM user_session_songs WHERE sessionId = 182 AND userId = 10"""
results = session.execute(query, timeout=None)
df2 = results._current_rows
df2[['artist', 'song', 'firstname', 'lastname']].head()


,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [16]:
# create table
cql("""DROP TABLE IF EXISTS user_session_songs; """)
cql("""CREATE TABLE user_session_songs (
    sessionId INT,
    userId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY ((sessionId), itemInSession))
    """)


# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_session_songs (sessionId, userId, itemInSession, artist, song, firstName, lastName)
                VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

        
# validate data
session.row_factory = pandas_factory
session.default_fetch_size = None
query = """SELECT sessionId, userId, itemInSession, artist, song, firstName, lastName
        FROM user_session_songs WHERE sessionId = 182"""
results = session.execute(query, timeout=None)
df2 = results._current_rows
df2[['artist', 'song', 'firstname', 'lastname']].head()


,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Translation in to CQL:
* `SELECT * FROM song_popularity WHERE song = 'All Hands Against His Own' `

* with `song` as partition key and `userId` as clustering column

Description of the query
* `song` is included in the WHERE statement therefore it must be part of the PRIMARY KEY   
* `song` alone does not make the primary key unique --> additional columns needed
    * `userId` would make more specific but a user can listen to the same song in different sessions
    * `sessionId, itemInSession` would make it unique as a sessionId has only one distinct user   
      and it would stay unique even if that user wants to hear a song twice in that session
* therefore will `song` serve as partition key
* and `sessionId, itemInSession` serve as clustering columns

* The result has to be ordered by `itemInSession`, therefore this will become the clustering column
* `sessionId, userId, itemInSession` make a row unique --> no additional columns needed for the PRIMARY KEY
* Again, if only `sessionId` would be partition key that could lead to an uneven distribution as some sessions may be longer than others
* Therefore `sessionId` and `userId` together serve as partition key

* `sessionId, userId, itemInSession` must come first in the CREATE, INSERT and SELECT statements,   
  as otherwise it will have consequences for the READ performance

In [9]:
# create table
cql("""DROP TABLE IF EXISTS song_popularity; """)
cql("""CREATE TABLE song_popularity (
    song TEXT,
    sessionId INT,
    itemInSession INT,
    lastName TEXT,
    firstName TEXT,
    PRIMARY KEY (song, sessionId, itemInSession)
    )""")

# insert data
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_popularity (song, sessionId, itemInSession, lastName, firstName)
                VALUES (%s, %s, %s, %s)"""
        session.execute(query, (line[9], int(line[10]), line[4], line[1]))

# validate data
session.row_factory = pandas_factory
session.default_fetch_size = None
query = "SELECT * FROM song_popularity WHERE song = 'All Hands Against His Own' "
results = session.execute(query, timeout=None)
df3 = results._current_rows
df3[['firstname', 'lastname']].head()


,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Dropping the tables and closing the session

In [1]:
cql("""DROP TABLE IF EXISTS session_songs; """)
cql("""DROP TABLE IF EXISTS user_session_songs; """)
cql("""DROP TABLE IF EXISTS song_popularity; """)
session.shutdown()
cluster.shutdown()

NameError: name 'cql' is not defined